In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign, realign_final
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from pycpd import RigidRegistration, DeformableRegistration
import open3d as o3d
from cycpd import rigid_registration

In [13]:
plate = 5
for j in range(0,7):
    my_file = open("/scratch/shared/mrozemul/Fiji.app/align.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
    my_file.write(f'for i in `seq {8*j} {8*j+7}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/final_alignment.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/align.sh
    !sbatch /scratch/shared/mrozemul/Fiji.app/align.sh

Submitted batch job 9365975
Submitted batch job 9365976
Submitted batch job 9365977
Submitted batch job 9365978
Submitted batch job 9365979
Submitted batch job 9365980
Submitted batch job 9365981


In [2]:
plate = 5
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
len(list_dir_interest)

54

In [ ]:
plt.close('all')
plate = 5
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[5:50]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
Rs=[]
ts=[]
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
    transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
    R,t = transform['R'],transform['t']
    Rs.append(R)
    ts.append(t)
for j in range(22,32): 
    skeleton1,skeleton2 = skel_docs[j-1],skel_docs[j]
    R,t = Rs[j],ts[j]
    skelet_pos = np.array(list(skeleton1.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    X = np.transpose(skelet_pos[samples,:])
    skelet_pos = np.array(list(skeleton2.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    Y = np.transpose(skelet_pos[samples,:])
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
    ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
    ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
for i,(x,y) in enumerate (zip(ts,Rs)):
    print(i,x,y,dates[i])

0 [[-18.81602709   3.21275639]] [[ 9.99999797e-01  6.37328984e-04]
 [-6.37328984e-04  9.99999797e-01]] 0715_0734
1 [[ 14.35258654 -30.76979784]] [[ 9.99999941e-01 -3.43262452e-04]
 [ 3.43262452e-04  9.99999941e-01]] 0715_1134
2 [[13.87952994 36.51369895]] [[ 9.99999914e-01 -4.14264948e-04]
 [ 4.14264948e-04  9.99999914e-01]] 0715_1534
3 [[-36.6249105  -12.83933703]] [[ 0.99999931  0.00117363]
 [-0.00117363  0.99999931]] 0715_1934
4 [[ 3.89138625 33.11108382]] [[ 9.99999767e-01 -6.82361819e-04]
 [ 6.82361819e-04  9.99999767e-01]] 0715_2334
5 [[18.45238354 -1.34930289]] [[ 9.99999978e-01  2.08946349e-04]
 [-2.08946349e-04  9.99999978e-01]] 0716_0334
6 [[-21.09412467 -32.77767644]] [[ 1.00000000e+00  1.67060444e-05]
 [-1.67060444e-05  1.00000000e+00]] 0716_0734
7 [[-15.9600903   20.23312695]] [[ 9.99999897e-01  4.54099873e-04]
 [-4.54099873e-04  9.99999897e-01]] 0716_1134
8 [[ 1774.36936321 -1144.34063749]] [[ 9.99999953e-01 -3.06474866e-04]
 [ 3.06474866e-04  9.99999953e-01]] 0718_0925
9

In [28]:
plate = 9
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[0:12]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
# for j in range(8): 
#     skeleton1,skeleton2 = skel_docs[j],skel_docs[j+1]


OSError: The file /scratch/shared/mrozemul/Fiji.app/20200714_1415_Plate09/Analysis/skeleton.mat does not exist.

In [25]:
dates_datetime_chosen

[datetime.datetime(2020, 7, 16, 11, 34), datetime.datetime(2020, 7, 18, 9, 25)]

In [11]:
len(skeleton2.keys())/100

13430.87

In [13]:
# skelet_pos = np.array(list(skeleton1.keys()))
# samples = np.random.choice(skelet_pos.shape[0],len(skeleton2.keys())//100)
# X = np.transpose(skelet_pos[samples,:])
# skelet_pos = np.array(list(skeleton2.keys()))
# samples = np.random.choice(skelet_pos.shape[0],len(skeleton2.keys())//100)
# Y = np.transpose(skelet_pos[samples,:])

In [17]:
factor = 10
shape=skels[0].shape
final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
for pixel in skeleton1.keys():
    x=pixel[0]//factor
    y=pixel[1]//factor
    final_picture[x,y]+=1
skelet_posX = np.transpose(np.array(final_picture.nonzero()))
final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
for pixel in skeleton2.keys():
    x=pixel[0]//factor
    y=pixel[1]//factor
    final_picture[x,y]+=1
skelet_posY = np.transpose(np.array(final_picture.nonzero()))
samples = np.random.choice(skelet_posX.shape[0],len(skeleton2.keys())//(100*factor))
X = np.transpose(skelet_posX[samples,:])
samples = np.random.choice(skelet_posY.shape[0],len(skeleton2.keys())//(100*factor))
Y = np.transpose(skelet_posY[samples,:])

In [19]:
len(skeleton2.keys())//(100*factor)

1343

In [18]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
reg = rigid_registration(**{'X': np.transpose(X.astype(float)), 'Y': np.transpose(Y.astype(float)),'scale': False})
out = reg.register()
Rfound = reg.R[0:2,0:2]
tfound= np.dot(Rfound,reg.t[0:2])

Iteration:1
ML:123648.739; 	ML change (error):123648.739; 	Sigma^2:25057519.865; 	Sigma^2 change:49417356.085
[                                                                        ]
Iteration:2
ML:114967.883; 	ML change (error):  8680.857; 	Sigma^2:15019372.376; 	Sigma^2 change:10038147.489
[=                                                                       ]
Iteration:3
ML:112493.617; 	ML change (error):  2474.265; 	Sigma^2:10451663.675; 	Sigma^2 change:4567708.701
[==                                                                      ]
Iteration:4
ML:111106.969; 	ML change (error):  1386.649; 	Sigma^2:7734724.031; 	Sigma^2 change:2716939.644
[==                                                                      ]
Iteration:5
ML:110101.785; 	ML change (error):  1005.184; 	Sigma^2:5921703.400; 	Sigma^2 change:1813020.631
[===                                                                     ]
Iteration:6
ML:109290.723; 	ML change (error):   811.062; 	Sigma^2:4643206.033; 

In [16]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
Yrep=np.transpose(np.transpose(np.dot(Rfound,np.transpose(np.transpose(X))))-tfound)
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
nx_graph1,pos1 = generate_nx_graph(from_sparse_to_graph(skeleton1))
nx_graph2,pos2 = generate_nx_graph(from_sparse_to_graph(skeleton2))
pruned1 = prune_graph(nx_graph1)
pruned2 = prune_graph(nx_graph2)
t_init=-tfound
Rot_init= Rfound
X = np.transpose(np.array([pos1[node] for node in pruned1 if pruned1.degree(node)==3]))
Y = np.transpose(np.array([pos2[node] for node in pruned2 if pruned2.degree(node)==3]))
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(X[0,:],X[1,:])
ax.scatter(Y[0,:],Y[1,:])
Xex = np.transpose(np.transpose(np.dot(Rot_init,X))+t_init)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(Xex[0,:],Xex[1,:])
ax.scatter(Y[0,:],Y[1,:])
X = np.insert(X, 2, values=0, axis=0) 
Y = np.insert(Y, 2, values=0, axis=0) 
print(X.shape,Y.shape)
vectorX = o3d.utility.Vector3dVector(np.transpose(X))
vectorY = o3d.utility.Vector3dVector(np.transpose(Y))
source = o3d.geometry.PointCloud(vectorX)
target = o3d.geometry.PointCloud(vectorY)
threshold = 200
trans_init = np.asarray([[Rot_init[0,0], Rot_init[0,1], 0, t_init[0]],
                         [Rot_init[1,0], Rot_init[1,1], 0, t_init[1]],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
reg_p2p = o3d.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
Rfound = reg_p2p.transformation[0:2,0:2]
tfound = reg_p2p.transformation[0:2,3]
print(Rfound,tfound)
X,Y=X[0:2,:],Y[0:2,:]
Yrep=np.transpose(np.transpose(np.dot(Rfound,X))+tfound)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

[17118.00202]
[19579.561526]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(3, 9002) (3, 9859)
registration::RegistrationResult with fitness=9.571206e-01, inlier_rmse=2.850703e+01, and correspondence_set size of 8616
Access transformation to get result.
[[ 0.99928864  0.03771231]
 [-0.03771231  0.99928864]] [-3786.70690899  3884.95989254]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
sio.savemat(path_snap+'/Analysis/transform.mat',{'R' : Rfound,'t' : tfound})